In [2]:
import requests
import pandas as pd
import os
import wget
from urllib.error import HTTPError

In [3]:
root_folder = 'dados/'

folder_renach = 'renach/'
folder_infracoes = 'infracoes/'

#### Criando dados de data

In [67]:
data = pd.DataFrame(pd.date_range(start='01-01-2018', end='31-12-2022'), columns=['data'])

data['id'] = data.index + 1
data['ano'] = data['data'].dt.year
data['mes'] = data['data'].dt.month
data['ano_mes'] = data['ano'] * 100 + data['mes']

In [68]:
data

,data,id,ano,mes,ano_mes
0,2018-01-01,1,2018,1,201801
1,2018-01-02,2,2018,1,201801
2,2018-01-03,3,2018,1,201801
3,2018-01-04,4,2018,1,201801
4,2018-01-05,5,2018,1,201801
...,...,...,...,...,...
1821,2022-12-27,1822,2022,12,202212
1822,2022-12-28,1823,2022,12,202212
1823,2022-12-29,1824,2022,12,202212
1824,2022-12-30,1825,2022,12,202212


## Lendo dados dos arquivos do RENACH

In [ ]:
os.chdir(root_folder + folder_renach)

for ano in [2018, 2019, 2020, 2021, 2022]:
    for mes in range(1, 13):    
        
        try:
            data = f'{ano}_{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renach/csv/condutores_habilitados_{data}.csv'
            
            wget.download(url)
        except:
            data = f'{ano}-{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renach/csv/condutores_habilitados_{data}.csv'

            wget.download(url)

os.chdir('../..')            

In [ ]:
df_result = pd.DataFrame()

for file in os.listdir(root_folder + folder_renach):
    df = pd.read_csv(root_folder + folder_renach + file, encoding='utf_16', decimal=',', thousands='.')

    file = file.replace('-', '_')  
    df['ano_mes'] = int(file.split('_')[2] + file.split('_')[3].split('.')[0])
    df_result = pd.concat([df_result, df], axis='index')

### Adiquirindo a relação de nome/sigla das UFs

In [ ]:
response = requests.get('http://servicodados.ibge.gov.br/api/v1/localidades/estados?orderBy=id').json()

In [ ]:
ufs = {}
for uf in response: ufs[uf['nome']] = uf['sigla']

In [ ]:
df_result['uf'] = [ufs[nome_uf] for nome_uf in df_result['UF Habilitação Atual']]
df_result['categoria_cnh'] = df_result['Categoria'].str.split('-', expand=True)[0]

In [ ]:
df_result = df_result[['uf', 'Sexo', 'Faixa Etária', 'categoria_cnh', 'Qt. Condutor Histórico', 'ano_mes']]

quantidade_condutores = df_result.rename(columns={
    'Sexo':'sexo',
    'Faixa Etária':'faixa_etaria',
    'Qt. Condutor Histórico':'qtd_condutores'
})

In [ ]:
quantidade_condutores

## Adiquirindo dados sobre os tipos de infrações

In [ ]:
os.chdir(root_folder + folder_infracoes)

try:
    url = 'https://www.gov.br/transportes/pt-br/centrais-de-conteudo/tabela-codigo-infracoes-renainf-xlsx'
    
    wget.download(url)
except:
    print('Erro de download.')

os.chdir('../..')  

In [60]:
df_infracoes = pd.read_excel(root_folder + folder_infracoes + 'codigos/tabela-codigo-infracoes-renainf-xlsx.xlsx', engine='openpyxl')

In [61]:
df_infracoes['id'] = df_infracoes.index + 1
df_infracoes = df_infracoes[['id', 'Código da Infração', 'Descrição da Infração', 'Gravidade', 'Órgão Competente']]

infracoes = df_infracoes.rename(columns={
    'Código da Infração':'codigo_infracao',
    'Descrição da Infração':'descricao_infracao',
    'Gravidade':'gravidade',
    'Órgão Competente':'orgao_competente'
})

In [63]:
infracoes

258

## Adiquirindo dados do RENAEST

#### Localidade

In [5]:
df_localidade = pd.read_csv('dados/renaest/Localidade_DadosAbertos_20230912.csv', sep=';')

In [6]:
df_localidade.head()

,chv_localidade,ano_referencia,mes_referencia,mes_ano_referencia,regiao,uf,codigo_ibge,municipio,regiao_metropolitana,qtde_habitantes,frota_total,frota_circulante
0,AC1200708201801,2018,1,12018,NORTE,AC,1200708,XAPURI,nao,18174,3956,2824
1,AC1200807201801,2018,1,12018,NORTE,AC,1200807,PORTO ACRE,nao,17459,3764,2697
2,AC1200351201801,2018,1,12018,NORTE,AC,1200351,MARECHAL THAUMATURGO,nao,17897,313,206
3,AC1200328201801,2018,1,12018,NORTE,AC,1200328,JORDAO,nao,7858,101,58
4,AC1200138201801,2018,1,12018,NORTE,AC,1200138,BUJARI,nao,9664,2168,1587


In [7]:
df_localidade.drop_duplicates(subset=['mes_ano_referencia', 'codigo_ibge'], inplace=True)
df_localidade = df_localidade[df_localidade['ano_referencia'] < 2023]

df_localidade['ano_mes'] = df_localidade['ano_referencia'] * 100 + df_localidade['mes_referencia']
df_localidade['metropolitana'] = df_localidade['regiao_metropolitana'] == 'sim'

df_localidade = df_localidade.rename(columns={
    'qtde_habitantes':'qtd_habitantes'
})

In [8]:
localidade = df_localidade[['ano_mes', 'municipio', 'uf', 'metropolitana', 'qtd_habitantes', 'frota_total', 'frota_circulante', 'regiao', 'codigo_ibge']].reset_index(drop=True)
localidade['id'] = localidade.index + 1

In [9]:
localidade

,ano_mes,municipio,uf,metropolitana,qtd_habitantes,frota_total,frota_circulante,regiao,codigo_ibge,id
0,201801,XAPURI,AC,False,18174,3956,2824,NORTE,1200708,1
1,201801,PORTO ACRE,AC,False,17459,3764,2697,NORTE,1200807,2
2,201801,MARECHAL THAUMATURGO,AC,False,17897,313,206,NORTE,1200351,3
3,201801,JORDAO,AC,False,7858,101,58,NORTE,1200328,4
4,201801,BUJARI,AC,False,9664,2168,1587,NORTE,1200138,5
...,...,...,...,...,...,...,...,...,...,...
334255,202212,TAIPAS DO TOCANTINS,TO,False,2193,394,289,NORTE,1720937,334256
334256,202212,FIGUEIROPOLIS,TO,False,5182,2672,1890,NORTE,1707652,334257
334257,202212,MATEIROS,TO,False,2810,532,365,NORTE,1712702,334258
334258,202212,NOVA OLINDA,TO,False,12075,3898,3001,NORTE,1714880,334259


#### Veículos

In [65]:
df_veiculo = pd.read_csv('dados/renaest/TipoVeiculo_DadosAbertos_20230912.csv', sep=';')

veiculo = df_veiculo.copy()

In [66]:
veiculo.drop_duplicates(subset=['tipo_veiculo'], inplace=True)
veiculo = veiculo[['tipo_veiculo']].reset_index(drop=True)
veiculo['id'] = veiculo.index + 1

In [67]:
veiculo = veiculo.rename(columns={
    'tipo_veiculo':'tipo'
})

In [69]:
veiculo

,tipo,id
0,NAO INFORMADO,1
1,AUTOMOVEL,2
2,BICICLETA,3
3,BONDE,4
4,CAMINHAO,5
5,CAMINHAO-TRATOR,6
6,CAMINHONETE,7
7,CAMIONETA,8
8,CARRO DE MAO,9
9,CARROCA,10


#### Vítimas


In [15]:
df_vitimas = pd.read_csv('dados/renaest/Vitimas_DadosAbertos_20230912.csv', sep=';')

In [16]:
df_vitimas = df_vitimas[['num_acidente', 'faixa_idade', 'genero', 'tp_envolvido', 'susp_alcool']].reset_index(drop=True)
df_vitimas['id'] = df_vitimas.index + 1

In [17]:
vitimas = df_vitimas.rename(columns={
    'tp_envolvido':'tipo',
    'susp_alcool':'suspeita_alcoolizado'
})

In [18]:
vitimas

,num_acidente,faixa_idade,genero,tipo,suspeita_alcoolizado,id
0,1003418,ENTRE 40 E 44 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,1
1,1473559,ENTRE 25 E 29 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,2
2,254169,ENTRE 25 E 29 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,3
3,352762,ENTRE 25 E 29 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,4
4,3248613,ENTRE 25 E 29 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,5
...,...,...,...,...,...,...
7835124,809551,ENTRE 18 E 24 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,7835125
7835125,2146372,ENTRE 18 E 24 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,7835126
7835126,5002149,ENTRE 65 E 69 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,7835127
7835127,2837727,ENTRE 45 E 49 ANOS,MASCULINO,MOTORISTA,NAO INFORMADO,7835128


#### Acidentes

In [70]:
df_acidentes = pd.read_csv('dados/renaest/Acidentes_DadosAbertos_20230912.csv', sep=';')

/tmp/ipykernel_29536/1315882353.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_acidentes = pd.read_csv('dados/renaest/Acidentes_DadosAbertos_20230912.csv', sep=';')


In [71]:
df_acidentes.head()

,num_acidente,chv_localidade,data_acidente,uf_acidente,ano_acidente,mes_acidente,mes_ano_acidente,codigo_ibge,dia_semana,fase_dia,...,lim_velocidade,tp_pista,ind_guardrail,ind_cantcentral,ind_acostamento,qtde_acidente,qtde_acid_com_obitos,qtde_envolvidos,qtde_feridosilesos,qtde_obitos
0,2448623,AC1200401201801,2018-01-24,AC,2018,1,12018,1200401,QUARTA-FEIRA,MANHA,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,3,3,0
1,3477494,AC1200385201801,2018-01-06,AC,2018,1,12018,1200385,SABADO,TARDE,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,1,1,0
2,2939219,AC1200401201801,2018-01-04,AC,2018,1,12018,1200401,QUINTA-FEIRA,MANHA,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,2,2,0
3,17595,AC1200203201801,2018-01-22,AC,2018,1,12018,1200203,SEGUNDA-FEIRA,NOITE,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,2,2,0
4,58010,AC1200401201801,2018-01-21,AC,2018,1,12018,1200401,DOMINGO,MANHA,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,2,2,0


In [72]:
df_acidentes['ano_mes'] = df_acidentes['ano_acidente'] * 100 + df_acidentes['mes_acidente']
df_acidentes['data_acidente'] = pd.to_datetime(df_acidentes['data_acidente'])

In [73]:
df_acidentes = df_acidentes.merge(
    localidade,
    how='inner',
    on=['codigo_ibge', 'ano_mes']
)

In [74]:
df_acidentes.rename(columns={'id':'id_localidade'}, inplace=True)

In [75]:
df_acidentes = df_acidentes.merge(
    vitimas,
    how='inner', 
    on=['num_acidente']
)

In [76]:
df_acidentes.rename(columns={'id':'id_vitima'}, inplace=True)

In [77]:
df_acidentes = df_acidentes[['num_acidente', 'id_localidade', 'id_vitima', 'data_acidente', 'tp_acidente', 'cond_meteorologica', 'qtde_envolvidos', 'qtde_feridosilesos', 'qtde_obitos']]

In [79]:
df_acidentes = df_acidentes.merge(
    df_veiculo,
    how='inner',
    on=['num_acidente']
).merge(
    veiculo,
    how='inner',
    left_on=['tipo_veiculo'],
    right_on=['tipo']
)

In [81]:
df_acidentes = df_acidentes[['num_acidente', 'id_localidade', 'id', 'id_vitima', 'data_acidente', 'tp_acidente', 'cond_meteorologica', 'qtde_envolvidos', 'qtde_feridosilesos', 'qtde_obitos']]

In [82]:
df_acidentes = df_acidentes.merge(
    data,
    how='inner',
    left_on=['data_acidente'],
    right_on=['data']
)

In [83]:
df_acidentes = df_acidentes[['num_acidente', 'id_localidade', 'id_x', 'id_vitima', 'id_y', 'data_acidente', 'tp_acidente', 'cond_meteorologica', 'qtde_envolvidos', 'qtde_feridosilesos', 'qtde_obitos']]

In [84]:
acidentes = df_acidentes.rename(columns={
    'id_x':'id_veiculo',
    'id_y':'id_data',
    'tp_acidente':'tipo_acidente',
    'qtde_feridosilesos':'qtde_feridos_ilesos'
})

In [85]:
acidentes

,num_acidente,id_localidade,id_veiculo,id_vitima,id_data,data_acidente,tipo_acidente,cond_meteorologica,qtde_envolvidos,qtde_feridos_ilesos,qtde_obitos
0,2448623,13,15,141,24,2018-01-24,COLISAO,CLARO,3,3,0
1,2448623,13,15,162,24,2018-01-24,COLISAO,CLARO,3,3,0
2,2448623,13,15,204,24,2018-01-24,COLISAO,CLARO,3,3,0
3,4085702,13,15,35,24,2018-01-24,COLISAO,CLARO,2,2,0
4,4085702,13,15,40,24,2018-01-24,COLISAO,CLARO,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...
9115216,4284772,185971,24,4402191,1062,2020-11-27,NAO INFORMADO,NAO INFORMADO,2,2,0
9115217,755078,309795,25,7185267,1062,2020-11-27,DESCONHECIDO,DESCONHECIDAS,1,1,0
9115218,2856568,309199,25,7181985,1062,2020-11-27,COLISAO LATERAL,CLARO,2,2,0
9115219,2856568,309199,25,7189156,1062,2020-11-27,COLISAO LATERAL,CLARO,2,2,0


## Adiquirindo os dados de quantidades de infrações

In [9]:
os.chdir(root_folder + folder_infracoes)

for ano in [2019, 2020, 2021, 2022]:    
    for mes in range(1, 13):    
        
        try:
            data = f'{ano}_{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renainf/xlsx/{data}_infracoes_com_np.xlsx'
            
            wget.download(url)
        
        except HTTPError as error:
            
            try:
                data = f'{ano}-{mes:02d}'
                url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renainf/xlsx/{data}_infracoes_com_np.xlsx'

                wget.download(url)
            except:
                print(error)
                print(f'{data} não presente.')
                pass

os.chdir('../..')            

HTTP Error 404: Not Found
2020-07 não presente.


In [52]:
def processa_cabecalho_tipo_1(path):    
    df = pd.read_excel(path, engine='openpyxl')

    file = path.split('/')[-1]
    df.columns = df.iloc[2]
    df = df.iloc[4:-1]

    # recupera o ano_mes que está sendo analisado
    file = file.replace('-', '_')  
    ano_mes = int(file.split('_')[0] + file.split('_')[1]) 

    # coloca o df no formato desejado para obter os dados
    df = df.reset_index(drop=True)
    df.columns.values[0] = 'codigo_infracao'
        
    df.set_index('codigo_infracao', inplace=True)

    return df, ano_mes

def processa_cabecalho_tipo_2(path):
    df = pd.read_excel(path, engine='openpyxl')

    file = path.split('/')[-1]
    
    # recupera o ano_mes que está sendo analisado
    file = file.replace('-', '_')  
    ano_mes = int(file.split('_')[0] + file.split('_')[1]) 
    
    df.columns = df.iloc[4]
    # trata última linha do arquivo que é usada como totalizador
    if ano_mes <= 202009:
        df = df.iloc[6:-1]
    else:
        df = df.iloc[6:]

    # coloca o df no formato desejado para obter os dados
    df = df.reset_index(drop=True)
    df.columns.values[0] = 'codigo_infracao'
        
    df.set_index('codigo_infracao', inplace=True)

    return df, ano_mes

def processa_cabecalho_tipo_3(path):
    df = pd.read_excel(path, engine='openpyxl')

    file = path.split('/')[-1]

    # recupera o ano_mes que está sendo analisado
    file = file.replace('-', '_')  
    ano_mes = int(file.split('_')[0] + file.split('_')[1]) 

    df.rename(columns={
        'UF':'uf',
        'Codigo_Infracao':'codigo_infracao',
        'Cod_Infracao':'codigo_infracao',
        'Quantidade':'qtd'
    }, inplace=True)
        
    df['ano_mes'] = ano_mes

    return df

def processa_cabecalho_tipo_4(path):
    df = pd.read_excel(path, engine='openpyxl')

    file = path.split('/')[-1]

    df = df[1:]

    # recupera o ano_mes que está sendo analisado
    file = file.replace('-', '_')  
    ano_mes = int(file.split('_')[0] + file.split('_')[1]) 

    # coloca o df no formato desejado para obter os dados
    df = df.reset_index(drop=True)
    df.columns.values[0] = 'codigo_infracao'
        
    df.set_index('codigo_infracao', inplace=True)

    return df, ano_mes

def processa_cabecalho_tipo_5(path):
    df = pd.read_excel(path, engine='openpyxl')

    file = path.split('/')[-1]

    df = df[1:]

    # recupera o ano_mes que está sendo analisado
    file = file.replace('-', '_')  
    ano_mes = int(file.split('_')[0] + file.split('_')[1]) 

    # coloca o df no formato desejado para obter os dados
    df.columns.values[0] = 'codigo_infracao'
        
    df.set_index('codigo_infracao', inplace=True)

    return df, ano_mes


In [53]:
def formata_df(df, ano_mes):
    # para cada arquivo, adiciona ano_mes e modifica o formato do df
    df_temp = pd.DataFrame()

    for idx in range(len(df.index)): #27 estados

        df2 = pd.DataFrame(data={
            'uf':df.columns,
            'qtd':df.iloc[idx].tolist(),
            'codigo_infracao':df.iloc[idx].name,
            'ano_mes':ano_mes
        })

        df_temp = pd.concat([df_temp, df2], axis='index')

    # guarda os valores no df final
    df_temp.dropna(subset=['qtd'], inplace=True)

    return df_temp

In [54]:
df_result = pd.DataFrame()
df_temp = pd.DataFrame()

folder_path = root_folder + folder_infracoes + 'cabecalho_1/'
for file in os.listdir(folder_path):

    df = pd.DataFrame()

    df, ano_mes = processa_cabecalho_tipo_1(folder_path + file)
    df_temp = pd.concat([df_temp, formata_df(df, ano_mes)], axis='index')

df_result = pd.concat([df_result, df_temp], axis='index')

folder_path = root_folder + folder_infracoes + 'cabecalho_2/'
for file in os.listdir(folder_path):

    df = pd.DataFrame()

    df, ano_mes = processa_cabecalho_tipo_2(folder_path + file)
    df_temp = pd.concat([df_temp, formata_df(df, ano_mes)], axis='index')

df_result = pd.concat([df_result, df_temp], axis='index')

folder_path = root_folder + folder_infracoes + 'cabecalho_3/'
for file in os.listdir(folder_path):

    df = pd.DataFrame()

    df = processa_cabecalho_tipo_3(folder_path + file)
    df_temp = pd.concat([df_temp, df], axis='index')

df_result = pd.concat([df_result, df_temp], axis='index')

folder_path = root_folder + folder_infracoes + 'cabecalho_4/'
for file in os.listdir(folder_path):

    df = pd.DataFrame()

    df, ano_mes = processa_cabecalho_tipo_4(folder_path + file)
    df_temp = pd.concat([df_temp, formata_df(df, ano_mes)], axis='index')

df_result = pd.concat([df_result, df_temp], axis='index')

folder_path = root_folder + folder_infracoes + 'cabecalho_5/'
for file in os.listdir(folder_path):

    df = pd.DataFrame()

    df, ano_mes = processa_cabecalho_tipo_5(folder_path + file)
    df_temp = pd.concat([df_temp, formata_df(df, ano_mes)], axis='index')

df_result = pd.concat([df_result, df_temp], axis='index')

/home/lucas/.miniconda3/envs/siad/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/lucas/.miniconda3/envs/siad/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/lucas/.miniconda3/envs/siad/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/lucas/.miniconda3/envs/siad/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/lucas/.miniconda3/envs/sia

In [72]:
quantidade_infracoes = df_result.merge(
    infracoes,
    how='left',
    on='codigo_infracao'
)

In [74]:
quantidade_infracoes.rename(columns={
    'id':'id_infracao'
}, inplace=True)

quantidade_infracoes = quantidade_infracoes[['id_infracao', 'ano_mes', 'uf', 'qtd']]

In [75]:
quantidade_infracoes

,id_infracao,ano_mes,uf,qtd
0,1.0,201903,ACRE,23.0
1,1.0,201903,ALAGOAS,102.0
2,1.0,201903,AMAZONAS,138.0
3,1.0,201903,AMAPA,26.0
4,1.0,201903,BAHIA,947.0
...,...,...,...,...
776293,NaN,202211,RORAIMA,5.0
776294,NaN,202211,SANTA CATARINA,2442.0
776295,NaN,202211,SAO PAULO,9050.0
776296,NaN,202211,SERGIPE,19.0
